In [134]:
import numpy as np
import pandas as pd

In [135]:
df=pd.read_csv('AMZN_stock_data.csv')
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1997-05-15 00:00:00-04:00,0.121875,0.125000,0.096354,0.097917,1443120000,0.0,0.0
1,1997-05-16 00:00:00-04:00,0.098438,0.098958,0.085417,0.086458,294000000,0.0,0.0
2,1997-05-19 00:00:00-04:00,0.088021,0.088542,0.081250,0.085417,122136000,0.0,0.0
3,1997-05-20 00:00:00-04:00,0.086458,0.087500,0.081771,0.081771,109344000,0.0,0.0
4,1997-05-21 00:00:00-04:00,0.081771,0.082292,0.068750,0.071354,377064000,0.0,0.0
...,...,...,...,...,...,...,...,...
6982,2025-02-14 00:00:00-05:00,229.199997,229.889999,227.229996,228.679993,27031100,0.0,0.0
6983,2025-02-18 00:00:00-05:00,228.820007,229.300003,223.720001,226.649994,42975100,0.0,0.0
6984,2025-02-19 00:00:00-05:00,225.520004,226.830002,223.710007,226.630005,28566700,0.0,0.0
6985,2025-02-20 00:00:00-05:00,224.779999,225.130005,221.809998,222.880005,30001700,0.0,0.0


In [136]:
df['Date']= pd.to_datetime(df['Date'])

/var/folders/n_/1hzw47553v1bvg9z8kct6rd40000gn/T/ipykernel_42429/2691600196.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date']= pd.to_datetime(df['Date'])


In [137]:
df['Close'].describe()

count    6987.000000
mean       40.685061
std        58.125169
min         0.069792
25%         2.110000
50%         9.021000
75%        73.006500
max       242.059998
Name: Close, dtype: float64

In [138]:
df['High'].describe()

count    6987.000000
mean       41.165619
std        58.787641
min         0.072396
25%         2.148250
50%         9.158500
75%        73.643497
max       242.520004
Name: High, dtype: float64

In [139]:
df['Close'] = (df['Close'] > df['Open']).astype(int)
df['Difference']= (df['Close']-df['Open']>0).astype(int)


In [140]:
final=df[['Close','Difference']]

In [141]:
final

,Close,Difference
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
6982,0,0
6983,0,0
6984,1,0
6985,0,0


In [142]:
# Step 1: Build state and observation label lists
states = sorted(final['Close'].unique()) 
observations = sorted(final['Difference'].unique())

In [143]:
def calculate_emission_matrix(observations_seq, states_seq, obs_symbols=None, state_symbols=None, smoothing=1):
    if obs_symbols is None:
        obs_symbols = sorted(set(observations_seq))
    if state_symbols is None:
        state_symbols = sorted(set(states_seq))
        
    obs_idx = {obs: i for i, obs in enumerate(obs_symbols)}
    state_idx = {state: i for i, state in enumerate(state_symbols)}

    num_states = len(state_symbols)
    num_obs = len(obs_symbols)

    # Initialize with smoothing
    emission_matrix = np.ones((num_states, num_obs)) * smoothing

    for s, o in zip(states_seq, observations_seq):
        if s in state_idx and o in obs_idx:  # safety check
            i = state_idx[s]
            j = obs_idx[o]
            emission_matrix[i, j] += 1

    # Normalize rows
    row_sums = emission_matrix.sum(axis=1, keepdims=True)
    emission_matrix = emission_matrix / row_sums

    return emission_matrix, state_symbols, obs_symbols

# Apply the corrected function
emit_prob, state_labels, obs_labels = calculate_emission_matrix(
    final['Difference'].values,  # observations
    final['Close'].values,       # hidden states
    observations,                # obs_symbols
    states                       # state_symbols
)

emit_prob

array([[9.99713795e-01, 2.86204923e-04],
       [8.84186446e-01, 1.15813554e-01]])

In [144]:
def calculate_transition_matrix(states_sequence, all_states=None, smoothing=0):
    if all_states is None:
        states = sorted(set(states_sequence))
    else:
        states = list(all_states)
    state_idx = {state:i for i, state in enumerate(states)}
    num_states = len(states)
    counts = np.zeros((num_states, num_states))
    for t in range(1, len(states_sequence)):
        i = state_idx[states_sequence[t-1]]
        j = state_idx[states_sequence[t]]
        counts[i, j] += 1
    if smoothing > 0:
        counts += smoothing
    row_sums = counts.sum(axis=1, keepdims=True)
    transition_matrix = np.divide(counts, row_sums, where=row_sums!=0)
    return transition_matrix, states

trans_prob, state_labels = calculate_transition_matrix(final['Difference'].values, states)
trans_prob

array([[0.9680948 , 0.0319052 ],
       [0.51980198, 0.48019802]])

In [145]:
class HiddenMarkovModel:
    """A Hidden markov model which takes Transition matrix and Emmission matrix as inputs"""

    def __init__(self, transition_matrix, emission_matrix, prior=None):
      self.transition_matrix = transition_matrix
      self.emission_matrix = emission_matrix
      self.prior = prior
      return None

    def emission_dist(self, ev):
      return self.emission_matrix[:, ev]

In [146]:
def forward_step(Hmm,trans, fv, ev):
  emission_dist = Hmm.emission_dist(ev)
  forward_inf = fv
  transition_matrix = trans

  nonNormSolution = emission_dist * np.array([[1,0],[0,1]]) @transition_matrix @ forward_inf
  alpha = nonNormSolution / np.sum(nonNormSolution)
  return alpha

In [147]:
def forward_algorithm(Hmm,observations):
    T = len(observations)

    alpha = np.zeros((T,2))

    alpha[0] = Hmm.prior * Hmm.emission_dist(observations[0])
    alpha[0] = alpha[0] / np.sum(alpha[0])

    for t in range(1,T):
        alpha[t] = forward_step(Hmm,Hmm.transition_matrix, alpha[t-1],observations[t])
    return alpha

In [148]:
obs=list(final['Difference']) 
start_prob = np.ones(len(states)) / len(states)  
Hmm = HiddenMarkovModel(trans_prob, emit_prob, prior=start_prob)
forward_algorithm(Hmm,obs).shape

(6987, 2)

In [149]:
N = len(states)
start_prob = np.ones(N) / N 

In [150]:
def backward_step(HMM, bv, ev):
  transition_matrix=HMM.transition_matrix
  backward_inference=bv
  emission_dist=HMM.emission_dist(ev)
  nonNormSol= transition_matrix @ np.diag(emission_dist)@ backward_inference.reshape(-1,1)
  result=nonNormSol/np.sum(nonNormSol)
  return result.flatten()

In [151]:
T=len(obs)
beta = np.zeros((T, 2))  
backward_inference = np.array([1.0, 1.0])
for t in reversed(range(T)):
    beta[t] = backward_inference.copy()  # store current message at correct time step
    backward_inference = backward_step(Hmm, backward_inference, ev=obs[t])  # move one step backward

# Optional: print to check
print("Backward matrix shape:", beta.shape)
print(beta)

Backward matrix shape: (6987, 2)
[[0.49961517 0.50038483]
 [0.46848385 0.53151615]
 [0.40432089 0.59567911]
 ...
 [0.51888647 0.48111353]
 [0.51334613 0.48665387]
 [1.         1.        ]]


In [152]:
def viterbi_algorithm(obs, states, start_prob, trans_prob, emit_prob):
    """
    obs: list or array of observations (as integer indices)
    states: list of state names or values
    start_prob: numpy array of shape (N,)
    trans_prob: numpy array of shape (N,N)
    emit_prob: numpy array of shape (N,M)

    Returns:
        best_path: list of most likely states (by label, not index)
        delta: np.ndarray of shape (T, N), max probability at each state, each t
        psi: np.ndarray of shape (T, N), backpointer for optimal path
    """
    N = len(states)
    T = len(obs)

    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)

    # Initialization
    delta[0, :] = start_prob * emit_prob[:, obs[0]]
    psi[0, :] = 0

    # Recursion
    for t in range(1, T):
        for j in range(N):
            # For each state j, find best prior state i
            seq_probs = delta[t-1, :] * trans_prob[:, j]
            psi[t, j] = np.argmax(seq_probs)
            delta[t, j] = np.max(seq_probs) * emit_prob[j, obs[t]]

    # Termination
    best_last_state = np.argmax(delta[T-1, :])
    best_path_idx = np.zeros(T, dtype=int)
    best_path_idx[T-1] = best_last_state

    # Path backtracking
    for t in range(T-2, -1, -1):
        best_path_idx[t] = psi[t+1, best_path_idx[t+1]]

    # Convert indices to state labels
    best_path = [states[i] for i in best_path_idx]

    return best_path, delta, psi


In [153]:
# Step 1: Encode observations from 'Difference' column
obs_seq = [obs_labels.index(v) for v in final['Difference'].values if v in obs_labels]

# Step 2: Set hidden states from 'Close' values
states = sorted(final['Close'].unique())  # or use `state_labels` if defined
N = len(states)
start_prob = np.ones(N) / N  # Uniform initial probabilities

# Step 3: Run Viterbi to decode most likely Close states from Difference observations
best_path, delta, psi = viterbi_algorithm(
    obs=obs_seq,
    states=states,
    start_prob=start_prob,
    trans_prob=trans_prob,
    emit_prob=emit_prob
)

# Step 4: Add decoded hidden states to DataFrame
final = final.iloc[:len(best_path)].copy()
final['predicted_close'] = best_path

# Optional: If your Close values were binarized or labeled, map back to original labels
# Example:
# final['predicted_close_label'] = final['predicted_close'].map({0: 'low', 1: 'high'})

# Step 5: Display results
print(final[['Close', 'Difference', 'predicted_close']].to_string(index=False))

 Close  Difference  predicted_close
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     1           1                1
     1           1                1
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     1           1                1
     1           1                1
     1           1                1
     0           0                1
     0           0                1
     1           1                1
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     0           0                0
     0           0          

In [154]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# True hidden states (e.g., High column)
true_states = final['Close'].values

# Predicted hidden states from Viterbi
predicted_states = best_path

# Basic metrics
accuracy = accuracy_score(true_states, predicted_states)
conf_matrix = confusion_matrix(true_states, predicted_states)
precision = precision_score(true_states, predicted_states, zero_division=0)
recall = recall_score(true_states, predicted_states, zero_division=0)
f1 = f1_score(true_states, predicted_states, zero_division=0)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Accuracy: 0.5069
Confusion Matrix:
[[3352  140]
 [3305  190]]
Precision: 0.5758
Recall:    0.0544
F1 Score:  0.0993
